<a href="https://colab.research.google.com/github/Aparnakhaleanwar/Capstone1_EDA/blob/main/CapStone1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd

In [12]:
df = pd.read_csv("/content/Hotel_Bookings_2.csv")

print(df.to_string()) 

            hotel  is_canceled  lead_time  arrival_date_year arrival_date_month  arrival_date_week_number  arrival_date_day_of_month  stays_in_weekend_nights  stays_in_week_nights  adults  children  babies meal country market_segment distribution_channel  is_repeated_guest  previous_cancellations  previous_bookings_not_canceled reserved_room_type assigned_room_type  booking_changes deposit_type  agent  company  days_in_waiting_list    customer_type     adr  required_car_parking_spaces  total_of_special_requests reservation_status reservation_status_date
0    Resort Hotel            0        342               2015               July                        27                          1                        0                     0       2         0       0   BB     PRT         Direct               Direct                  0                       0                               0                  C                  C                3   No Deposit    NaN      NaN                     0     

In [15]:
#verify data types of all columns
df.dtypes


hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                            int64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                             

In [30]:
df['agent']=df['agent'].astype(int)

In [31]:
df.dtypes



hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                            int64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                             

In [32]:
#for Agent Analysis, replaced NaN values by 0 to perform further investigation.
df['agent'] = df['agent'].fillna(0)

In [33]:
df


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,0,NaN,0,Transient,0.0,0,0,Check-Out,01-07-2015
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,0,NaN,0,Transient,0.0,0,0,Check-Out,01-07-2015
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,0,NaN,0,Transient,75.0,0,0,Check-Out,02-07-2015
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304,NaN,0,Transient,75.0,0,0,Check-Out,02-07-2015
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240,NaN,0,Transient,98.0,0,1,Check-Out,03-07-2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744,Resort Hotel,1,41,2015,July,31,27,1,3,2,...,No Deposit,240,NaN,0,Transient,164.0,0,0,Canceled,17-06-2015
745,Resort Hotel,0,35,2015,July,31,27,1,3,2,...,No Deposit,196,NaN,0,Transient,149.0,0,0,Check-Out,31-07-2015
746,Resort Hotel,0,28,2015,July,31,27,1,4,2,...,No Deposit,240,NaN,0,Transient,154.0,1,0,Check-Out,01-08-2015
747,Resort Hotel,0,77,2015,July,31,27,1,4,2,...,No Deposit,240,NaN,0,Transient,195.0,0,1,Check-Out,01-08-2015


In [34]:
#Find count of each agent to predict which agent has given maximum business which can be considered while providing commission to agent.
df['agent'].value_counts()

240    344
0       70
250     70
241     53
305     20
242     19
40      15
8       13
5       13
142     11
196     10
175     10
134     10
156     10
243     10
147      9
105      6
115      6
15       6
152      5
171      5
6        4
261      3
95       3
146      3
184      2
2        2
177      2
167      1
67       1
104      1
300      1
96       1
149      1
244      1
143      1
9        1
127      1
304      1
306      1
3        1
303      1
36       1
Name: agent, dtype: int64

In [24]:
#Company Analysis
#Find Unique companies
df['company'].unique()

array([ nan, 110., 113., 270., 178., 240., 154.])

In [26]:
#Find companies wise hotel count. so if we considered it's a chain of hotel associated with any company then this data will 
#show which company hotels are preferable.

df['company'].value_counts()

110.0    4
178.0    2
113.0    1
270.0    1
240.0    1
154.0    1
Name: company, dtype: int64